process video annotation export and do analysis

In [26]:
import pandas as pd

In [27]:
df = pd.read_csv("kopp_video_annotation_export.csv")
df = df[df["label"] == 1]
df.head()

,video_id,label,full_head (0-2 range; 0=little of head; 1=enough head; 2=full head),"kaputt(1=yes, 0=no)","severe Movement cam/pat (1=yes, 0=no)","forward_movement (1=yes, 0=no)","Quality(0=bad, 1=ok)","Usable (1=yes, 0=no)",comment
0,1000159621_1.mp4,1,1.0,NaN,1.0,NaN,0.6,NaN,movement at end
2,1005454012_1.mp4,1,1.0,NaN,1.0,1.0,1.0,0.0,NaN
6,1008442025_1.mp4,1,1.0,NaN,NaN,1.0,1.0,0.0,NaN
8,1010125904_2.mp4,1,1.0,NaN,NaN,NaN,1.0,1.0,NaN
12,1014634942_1.mp4,1,1.0,NaN,NaN,NaN,1.0,1.0,NaN


fill nan values

In [28]:
# column kaputt(1=yes, 0=no) filled with 0
df["kaputt(1=yes, 0=no)"][df["kaputt(1=yes, 0=no)"].isnull()] = 0
# severe Movement cam/pat (1=yes, 0=no) filled with 0
df["severe Movement cam/pat (1=yes, 0=no)"][df["severe Movement cam/pat (1=yes, 0=no)"].isnull()] = 0
# column forward_movement (1=yes, 0=no) filled with 0
df["forward_movement (1=yes, 0=no)"][df["forward_movement (1=yes, 0=no)"].isnull()] = 0
# values bigger 0 are set to 1
df["forward_movement (1=yes, 0=no)"][df["forward_movement (1=yes, 0=no)"] > 0] = 1
# column Usable (1=yes, 0=no) filled with 0
df["Usable (1=yes, 0=no)"][df["Usable (1=yes, 0=no)"].isnull()] = 0
# set nan in comment to empty string
df["comment"][df["comment"].isnull()] = ""



C:\Users\ly37xoco\AppData\Local\Temp\ipykernel_14920\2132394913.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["kaputt(1=yes, 0=no)"][df["kaputt(1=yes, 0=no)"].isnull()] = 0
C:\Users\ly37xoco\AppData\Local\Temp\ipykernel_14920\213239

### process comment column

In [29]:
df["comment"].value_counts()

comment
                         635
not finished              44
start missing             28
too much movement         25
earring                   17
                        ... 
nothing happening          1
fluid mouth                1
not sure if usable         1
earring, not finished      1
bolus hard to see          1
Name: count, Length: 71, dtype: int64

In [30]:
df["not_fully_swallowed (1=yes, 0=no)"] = 0
# if "not finished" is in the comment, set not_fully_swallowed to 1
df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("not finished", case=False)] = 1
df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("not swallowed", case=False)] = 1
df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("no swallows", case=False)] = 1
df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("swallow not finished", case=False)] = 1
df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("no real swallow", case=False)] = 1
df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("no real swallows", case=False)] = 1
df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("swallowed little", case=False)] = 1
df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("not completely swallowed", case=False)] = 1
df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("not swallowed everything", case=False)] = 1

df["swallow_beginning_missing (1=yes, 0=no)"] = 0
# if [start missing, no beginning, beginning missing] is in the comment, set swallow_beginning_missing to 1
df["swallow_beginning_missing (1=yes, 0=no)"][df["comment"].str.contains("start missing", case=False)] = 1
df["swallow_beginning_missing (1=yes, 0=no)"][df["comment"].str.contains("no beginning", case=False)] = 1
df["swallow_beginning_missing (1=yes, 0=no)"][df["comment"].str.contains("beginning missing", case=False)] = 1

df["bolus_already_present (1=yes, 0=no)"] = 0
# if [bolus present, bolus pressent, bolus already present] is in the comment, set bolus_already_present to 1
df["bolus_already_present (1=yes, 0=no)"][df["comment"].str.contains("bolus present", case=False)] = 1
df["bolus_already_present (1=yes, 0=no)"][df["comment"].str.contains("bolus pressent", case=False)] = 1
df["bolus_already_present (1=yes, 0=no)"][df["comment"].str.contains("bolus already present", case=False)] = 1

df["accessory_artefact (1=yes, 0=no)"] = 0
# if [earring, glasses, earrings, earing] is in the comment, set accessory_artefact to 1
df["accessory_artefact (1=yes, 0=no)"][df["comment"].str.contains("earring", case=False)] = 1
df["accessory_artefact (1=yes, 0=no)"][df["comment"].str.contains("glasses", case=False)] = 1
df["accessory_artefact (1=yes, 0=no)"][df["comment"].str.contains("earrings", case=False)] = 1
df["accessory_artefact (1=yes, 0=no)"][df["comment"].str.contains("earing", case=False)] = 1
df["accessory_artefact (1=yes, 0=no)"][df["comment"].str.contains("earning", case=False)] = 1



# name column for cases like "hard to see / hard to segment  / bolus barely visible / not visible / difficult to segment / bolus not visible / not much visible / very hard to see / narely seeable / bolus hard to see
df["hard_to_see (1=yes, 0=no)"] = 0
# if [hard to see, hard to segment, barely visible, not visible, difficult to segment, not much visible, very hard to see, narely seeable, hard to see] is in the comment, set hard_to_see to 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("hard to see", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("hard to segment", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("bolus barely visible", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("not visible", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("difficult to segment", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("bolus not visible", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("not much visible", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("very hard to see", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("narely seeable", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("barely seeable", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("bolus hard to see", case=False)] = 1
df["hard_to_see (1=yes, 0=no)"][df["comment"].str.contains("not sure if enough visible", case=False)] = 1


C:\Users\ly37xoco\AppData\Local\Temp\ipykernel_14920\2939968698.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["not_fully_swallowed (1=yes, 0=no)"][df["comment"].str.contains("not finished", case=False)] = 1
C:\Users\ly37xoco\AppData

### rename cols

In [31]:
# rename columns "label" and "kaputt(1=yes, 0=no)"
df = df.rename(columns={
    "label": "label",
    "kaputt(1=yes, 0=no)": "artifact_detected",
    "severe Movement cam/pat (1=yes, 0=no)": "severe_movement",
    "forward_movement (1=yes, 0=no)": "forward_movement",
    "Usable (1=yes, 0=no)": "is_usable",
    "Quality(0=bad, 1=ok)": "quality",
    "full_head (0-2 range; 0=little of head; 1=enough head; 2=full head)": "head_visibility",
    "comment": "comment",
    "not_fully_swallowed (1=yes, 0=no)": "incomplete_swallow",
    "swallow_beginning_missing (1=yes, 0=no)": "missing_swallow_start",
    "bolus_already_present (1=yes, 0=no)": "bolus_present_initially",
    "accessory_artefact (1=yes, 0=no)": "accessory_artifact",
    "hard_to_see (1=yes, 0=no)": "visibility_issue"
})


In [32]:
# create column "Use for dataset" where value is True if is_usable is 1 and other columns are 0
df["use_for_dataset"] = False
df["use_for_dataset"] = (df["is_usable"] == 1) & (df["artifact_detected"] == 0) & (df["severe_movement"] == 0) & (df["forward_movement"] == 0) & (df["incomplete_swallow"] == 0) & (df["missing_swallow_start"] == 0) & (df["bolus_present_initially"] == 0) & (df["accessory_artifact"] == 0) & (df["visibility_issue"] == 0)


In [33]:
df["use_for_dataset"].value_counts()

use_for_dataset
False    593
True     275
Name: count, dtype: int64

In [34]:
df = df[['video_id', 'use_for_dataset', 'is_usable', 'artifact_detected', 'severe_movement', 'forward_movement', 'incomplete_swallow', 'missing_swallow_start', 'bolus_present_initially', 'accessory_artifact', 'visibility_issue']]
df.head()

,video_id,use_for_dataset,is_usable,artifact_detected,severe_movement,forward_movement,incomplete_swallow,missing_swallow_start,bolus_present_initially,accessory_artifact,visibility_issue
0,1000159621_1.mp4,False,0.0,0.0,1.0,0.0,0,0,0,0,0
2,1005454012_1.mp4,False,0.0,0.0,1.0,1.0,0,0,0,0,0
6,1008442025_1.mp4,False,0.0,0.0,0.0,1.0,0,0,0,0,0
8,1010125904_2.mp4,True,1.0,0.0,0.0,0.0,0,0,0,0,0
12,1014634942_1.mp4,True,1.0,0.0,0.0,0.0,0,0,0,0,0


export

In [35]:
df.to_csv("AnnotationsKopp_cleaned.csv", index=False)

### Analysis of Annotations

In [ ]:
df.describe()

In [ ]:
df["is_usable"].value_counts()

In [ ]:
df["visibility_issue"].value_counts()

In [ ]:
df["artifact_detected"].value_counts()

In [ ]:
df[df["missing_swallow_start"] == 1].shape[0]

In [ ]:
df[df["incomplete_swallow"] == 1].shape[0]

In [ ]:
df[df["bolus_present_initially"] == 1].shape[0]

In [ ]:
# how many rows with missing swallow start and or bolus present initially and or incomplete swallow
df[(df["missing_swallow_start"] == 1) | (df["bolus_present_initially"] == 1) | (df["incomplete_swallow"] == 1)].shape[0]

In [ ]:
df[df["accessory_artifact"] == 1].shape[0]

In [ ]:

# how many rows do we have if is_usable is 1 and artifact_detected is 0 and incomplete_swallow is 0 and missing_swallow_start is 0 and bolus_present_initially is 0 and accessory_artifact is 0 and visibility_issue is 0
df[(df["is_usable"] == 1) & (df["artifact_detected"] == 0) & (df["incomplete_swallow"] == 0) & (df["missing_swallow_start"] == 0) & (df["bolus_present_initially"] == 0) & (df["accessory_artifact"] == 0) & (df["visibility_issue"] == 0)].shape[0]

In [ ]:
df[(df["incomplete_swallow"] == 1) | (df["missing_swallow_start"] == 1) | (df["bolus_present_initially"] == 1)].shape[0]

In [ ]:
# how many rows do we have if is_usable is 1 and artifact_detected is 0 and incomplete_swallow is 0 and missing_swallow_start is 0 and bolus_present_initially is 0 and accessory_artifact is 0 and visibility_issue is 0 and forward_movement is 0 and severe_movement is 0
df[(df["is_usable"] == 1) & (df["artifact_detected"] == 0) & (df["incomplete_swallow"] == 0) & (df["missing_swallow_start"] == 0) & (df["bolus_present_initially"] == 0) & (df["accessory_artifact"] == 0) & (df["visibility_issue"] == 0) & (df["forward_movement"] == 0) & (df["severe_movement"] == 0)].shape[0]


In [ ]:
# how many rows do we have if artifact_detected is 1 and or incomplete_swallow is 1 and or missing_swallow_start is 1 and or bolus_present_initially is 1 and or accessory_artifact is 1 and or visibility_issue is 1 and or severe_movement is 1 and or forward_movement is 1
df[(df["artifact_detected"] == 1) | (df["incomplete_swallow"] == 1) | (df["missing_swallow_start"] == 1) | (df["bolus_present_initially"] == 1) | (df["accessory_artifact"] == 1) | (df["visibility_issue"] == 1) | (df["severe_movement"] == 1) | (df["forward_movement"] == 1)].shape[0]